In [7]:
#using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra

m = readdlm("../data/PowerEU/ElistPowerEU.txt")

const dim0 = size(m);
M_int = Array{Int64,2}(dim0[1],2);
for i =1:dim0[1]
	for j=1:2
		M_int[i,j] = Int64(m[i,j]);
	end
end	

N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]);

A0 =spzeros(N,N);
A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N);
A0 = A0+A0';
A,p = largest_component(A0);

A0=0;p=0;m =0;
gc();

LoadError: MethodError: no method matching Matrix{Int64}(::Int64, ::Int64)
[0mClosest candidates are:
[0m  Matrix{T}([91m::UniformScaling[39m, ::Integer, [91m::Integer[39m) where T at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:471
[0m  Array{T, N}([91m::Missing[39m, ::Any...) where {T, N} at baseext.jl:32
[0m  Array{T, N}([91m::Nothing[39m, ::Any...) where {T, N} at baseext.jl:31
[0m  ...

In [3]:
d = sum(A,2);
s = sum(d);

r = 80;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[4.80143, 4.62062, 4.47457, 4.4701, 4.40547, 4.26694, 4.21332, 4.10447, 4.13072, 3.95979, 3.87881, 3.87168, 3.90373, 3.85064, 3.83691, 3.80114, 3.78494, 3.76119, 3.77156, 3.72184, 3.67629, 3.64175, 3.62803, 3.6374, 3.58693, 3.60177, 3.55039, 3.547, 3.51816, 3.5253, 3.47676, 3.47903, 3.47316, 3.46784, 3.44623, 3.45914, 3.43375, 3.41193, 3.44215, 3.39214, 3.38571, 3.39191, 3.37553, 3.35442, 3.32956, 3.31968, 3.31115, 3.3489, 3.31022, 3.3104, 3.30775, 3.2766, 3.27827, 3.23478, 3.26139, 3.23224, 3.21944, 3.22249, 3.20665, 3.16863, 3.18154, 3.12602, 3.14989, 3.14764, 3.11525, 3.12032, 3.102, 3.11132, 3.09945, 3.08049, 3.0888, 3.07891, 3.06057, 3.07149, 3.07123, 3.06309, 3.05447, 3.05093, 3.05514, 3.03936]

In [18]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:50]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 1
Number of communities: 54
Modularity: 0.86369464124091


In [5]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:80
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 0
Number of communities: 45
Modularity: 0.67999274367217
n: 20
